# **使用 LangGraph 和 Qwen 模型实现上下文工程四大策略**

### **目标**
本 Notebook 将作为一份详细的技术指南，演示如何使用 LangGraph 框架和通义千问（Qwen）模型，一步步实现“上下文工程”的四大核心策略。

四大策略包括：
1.  **写入 (Write):** 为智能体构建一个“外部大脑”（暂存区），以在长任务中保持状态。
2.  **选择 (Select):** 使用检索增强生成（RAG）从知识库中精准调取信息。
3.  **压缩 (Compress):** 智能地总结对话历史，以节省成本和Token。
4.  **隔离 (Isolate):** 使用多智能体（Multi-agent）架构，将复杂任务分解给专家处理。

---
### **第一步：环境设置与模型初始化**

In [1]:
! pip install langchain langchain-qwq langgraph pandas python-dotenv langchain_community dashscope faiss-cpu pandas

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://mirrors.aliyun.com/pypi/simple


In [2]:
import os
import getpass
import json
import operator
from typing import List, TypedDict, Annotated

import tiktoken
from langchain_core.messages import BaseMessage, HumanMessage, ToolMessage, SystemMessage
from langchain_core.tools import tool
from langchain_qwq import ChatQwen
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolNode

In [3]:
# --- 1. 设置API密钥 ---
# 请注意：这里需要的是 DashScope 的 API Key
if "DASHSCOPE_API_KEY" not in os.environ:
    os.environ["DASHSCOPE_API_KEY"] = getpass.getpass("请输入您的DashScope API Key: ")

In [4]:
# --- 2. 初始化Qwen模型 ---
# 我们将使用 qwen3-32b 模型作为我们智能体的“大脑”
try:
    model = ChatQwen(model="qwen3-30b-a3b", base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",  enable_thinking=False)
    print("Qwen 模型初始化成功！")
except Exception as e:
    print(f"模型初始化失败，请检查API Key或网络连接: {e}")

Qwen 模型初始化成功！


In [5]:
# --- 3. 初始化Token计算器 ---
encoding = tiktoken.get_encoding("cl100k_base")

## **策略一：写入 (Write) - 构建智能体的“草稿纸”**

**核心思想:** 不把所有中间步骤和思考都塞进主对话历史（`messages`），而是将它们“写入”到一个独立的“暂存区”（`scratchpad`）。这可以保持主对话的清晰，并为智能体提供一个可靠的短期记忆，防止在长任务中“失忆”。

**实现:** 我们将在`AgentState`中增加一个`scratchpad`字段，并通过添加`SystemMessage`来指导模型的行为，防止无限循环。

In [6]:
# --- 1. 定义状态 ---
from typing_extensions import TypedDict
from typing import List, Annotated
import operator

class ToolCallRecord(TypedDict):
    step: int
    tool_name: str
    args: dict
    result: str

class WriteStrategyState(TypedDict):
    messages: Annotated[list, operator.add]
    # 结构化 scratchpad，保留完整历史
    scratchpad: dict  # {"history": List[ToolCallRecord], "final_answer": str}

In [7]:
# --- 2. 定义工具 ---
from langchain_core.tools import tool

@tool
def simple_calculator(operation: str, a: int, b: int) -> int:
    """一个简单的计算器工具，执行加减乘除。"""
    if operation == "add":
        return a + b
    if operation == "subtract":
        return a - b
    if operation == "multiply":
        return a * b
    if operation == "divide" and b != 0:
        return a // b
    return "无效操作"

In [8]:
# --- 3. 定义图的节点 ---
from langgraph.prebuilt import ToolNode
from langchain_core.messages import SystemMessage, HumanMessage
from langgraph.graph import END

tools = [simple_calculator]
tool_node = ToolNode(tools)
model_with_tools = model.bind_tools(tools)

def agent_with_scratchpad(state: WriteStrategyState):
    """
    Agent 节点：决定下一步动作，并更新暂存区。
    """
    print("---AGENT NODE---")
    response = model_with_tools.invoke(state['messages'])

    if response.tool_calls:
        # 暂存当前待执行的工具
        state['scratchpad']['pending_tool'] = response.tool_calls[0]
        print(f"🧠 Agent Action: Call tool `{response.tool_calls[0]['name']}` "
              f"with arguments `{response.tool_calls[0]['args']}`")
    else:
        # 全部完成，保存最终答案
        state['scratchpad']['final_answer'] = response.content
        print(f"✅ Final Answer: {response.content}")

    return {"messages": [response], "scratchpad": state['scratchpad']}

def tool_node_with_scratchpad(state: WriteStrategyState):
    """
    Tool 节点：执行工具，并把结果记录到 history。
    """
    print("---TOOL NODE---")
    last_message = state['messages'][-1]
    tool_messages = tool_node.invoke([last_message])

    # 取出待处理的工具调用
    pending = state['scratchpad']['pending_tool']
    record = ToolCallRecord(
        step=len(state['scratchpad'].get("history", [])) + 1,
        tool_name=pending['name'],
        args=pending['args'],
        result=str(tool_messages[0].content)
    )
    # 追加到历史
    state['scratchpad'].setdefault("history", []).append(record)
    print(f"📝 Recorded Tool Call: {record}")
    state['scratchpad'].pop("pending_tool", None)  # 清理

    #print(f"🛠️ Tool Result: `{record['result']}`")
    return {"messages": tool_messages, "scratchpad": state['scratchpad']}

In [9]:
# --- 4. 构建图 ---
from langgraph.graph import StateGraph

write_graph_builder = StateGraph(WriteStrategyState)
write_graph_builder.add_node("agent", agent_with_scratchpad)
write_graph_builder.add_node("action", tool_node_with_scratchpad)
write_graph_builder.set_entry_point("agent")

# 条件边：检查是否还有未完成的工具
def should_continue(state: WriteStrategyState) -> str:
    # 若最终答案已存在，直接结束
    if state['scratchpad'].get("final_answer"):
        return END
    return "action"

write_graph_builder.add_conditional_edges("agent", should_continue, {"action": "action", END: END})
write_graph_builder.add_edge("action", "agent")
write_graph = write_graph_builder.compile()

In [10]:
# --- 5. 演示 ---
print("### 工程造价预算调整计算 ###")
task = (
    "1) 初始工程预算 128 万元与设计变更追加 72 万元进行合并；\n"
    "2) 合并后的预算按季度材料价格波动系数进行 3 倍调整；\n"
    "3) 调整后的预算因国际标准转换需除以 100 得到标准工料单位；\n"
    "4) 标准工料单位值按 20 倍风险系数扩大，形成最终风险预算；\n"
    "5) 从风险预算中扣除 22 万元的质量保证金。\n"
    "请列出每一步的数值结果，并以『工程最终预算：{数值}万元』的格式给出答案。"
)

system_prompt = (
    "你是一个工程造价计算助手。请按步骤使用 `simple_calculator` 工具进行工程预算计算。"
)
initial_messages = [
    SystemMessage(content=system_prompt),
    HumanMessage(content=task)
]
initial_state = {"messages": initial_messages, "scratchpad": {"history": [], "final_answer": None}}

# 使用 .stream() 观察每一步
for step in write_graph.stream(initial_state, {"recursion_limit": 20}):
    #print(step)
    print("---")

### 工程造价预算调整计算 ###
---AGENT NODE---
🧠 Agent Action: Call tool `simple_calculator` with arguments `{'a': 128, 'b': 72, 'operation': 'add'}`
---
---TOOL NODE---
📝 Recorded Tool Call: {'step': 1, 'tool_name': 'simple_calculator', 'args': {'a': 128, 'b': 72, 'operation': 'add'}, 'result': '200'}
---
---AGENT NODE---
🧠 Agent Action: Call tool `simple_calculator` with arguments `{'a': 200, 'b': 3, 'operation': 'multiply'}`
---
---TOOL NODE---
📝 Recorded Tool Call: {'step': 2, 'tool_name': 'simple_calculator', 'args': {'a': 200, 'b': 3, 'operation': 'multiply'}, 'result': '600'}
---
---AGENT NODE---
🧠 Agent Action: Call tool `simple_calculator` with arguments `{'a': 600, 'b': 100, 'operation': 'divide'}`
---
---TOOL NODE---
📝 Recorded Tool Call: {'step': 3, 'tool_name': 'simple_calculator', 'args': {'a': 600, 'b': 100, 'operation': 'divide'}, 'result': '6'}
---
---AGENT NODE---
🧠 Agent Action: Call tool `simple_calculator` with arguments `{'a': 6, 'b': 20, 'operation': 'multiply'}`
---
---TOO

## **策略二：选择 (Select) - 精准的"信息调取"**

**核心思想：** 使用RAG（检索增强生成）技术，从外部知识库中精准检索最相关的信息片段，只将必要信息注入上下文。

**实现步骤：**
1. 创建产品知识库（模拟向量数据库）
2. 构建RAG检索器
3. 设计智能体流程：问题 → 检索 → 生成答案

In [11]:
# --- 1. 创建模拟产品知识库 ---
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from langchain_community.embeddings import DashScopeEmbeddings

# 创建嵌入模型
embeddings = DashScopeEmbeddings(model="text-embedding-v3")

In [12]:
product_docs = [
    Document(page_content="""电力造价管理系统：
- 理念：电力工程全生命周期造价管理
- 覆盖专业：发电/变电/输电/农配网/运维检修五大领域
- 核心功能：支持可研估算、初步设计概算、施工图预算、招投标报价、竣工结算全流程
- 技术特点：内置行业计价依据与业务场景库
- 优势：智能高效完成造价编制，确保合规性与准确性
- 适用对象：电力工程造价咨询机构、建设单位""", 
             metadata={"product": "电力造价管理系统", "category": "造价软件"}),
    
    Document(page_content="""电力设计协同平台：
- 专业覆盖：输电/变电/配电三大核心专业
- 核心功能：预置设计规程规范、典型设计方案、标准物料库、成果模板
- 技术特点：可视化图形设计界面，支持数字化成果输出
- 优势：提升设计效率30%+，规范设计质量
- 适用场景：电力设计院、工程总承包项目""", 
             metadata={"product": "电力设计协同平台", "category": "设计软件"}),
    
    Document(page_content="""易数大数据分析平台：
- 核心能力：构建大数据可视化分析生态圈
- 功能模块：数据资产治理、多维度可视化分析、智能决策支持
- 适用领域：政府能源监管、电力企业运营分析、新能源项目管理
- 技术优势：低代码操作界面，支持TB级数据实时分析
- 部署方式：私有云/公有云双模式""", 
             metadata={"product": "易数大数据平台", "category": "数据分析"}),
    
    Document(page_content="""电力业务管理工具集：
- 核心产品：清标工具（招标文件智能审查）
           智慧招投标系统（全流程电子化）
- 功能亮点：自动识别风险项、智能生成审查报告
- 适用对象：业主单位/施工单位/招标代理机构
- 效率提升：人工成本降低50%，错误率下降80%""", 
             metadata={"product": "电力管理工具集", "category": "管理工具"}),
    
    # 行业知识补充文档
    Document(page_content="""电力工程采购指南：
1. 突出全生命周期成本优势
2. 强调合规性与行业标准适配
3. 技术参数需匹配国网/南网规范
4. 提供典型项目成功案例
5. 采用分阶段报价策略""", 
             metadata={"doc_type": "采购指南"}),
    
    Document(page_content="""电力行业用户需求：
- 采购决策关键因素：
  1. 系统合规性（82%）
  2. 数据安全性（76%） 
  3. 与现有系统兼容性（68%）
  4. 本地化服务能力（63%）
  5. ROI周期（55%）""", 
             metadata={"doc_type": "行业洞察"}),
]

# 创建向量数据库
vector_db = FAISS.from_documents(product_docs, embeddings)

In [13]:
# --- 2. 构建RAG检索器 ---
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

# 创建检索器
retriever = vector_db.as_retriever(search_kwargs={"k": 2})

def format_docs(docs):
    """格式化检索到的文档"""
    return "\n\n".join(f"## 来源 {i+1}:\n{doc.page_content}" for i, doc in enumerate(docs))

# 创建RAG提示模板
rag_template = """
你是一位专业的产品文案助手。请根据提供的背景信息回答用户问题。

<背景信息>
{context}
</背景信息>

用户问题：{question}

请用专业、简洁的语言回答，突出产品核心优势：
"""
rag_prompt = ChatPromptTemplate.from_template(rag_template)

# 创建RAG链
rag_chain = (
    {"context": retriever | RunnableLambda(format_docs), "question": RunnablePassthrough()}
    | rag_prompt
    | model
    | StrOutputParser()
)


In [14]:
# --- 3. 设计智能体流程 ---
class SelectStrategyState(TypedDict):
    messages: List[BaseMessage]
    context: str  # 存储检索到的上下文

def retrieve_context(state: SelectStrategyState):
    """检索节点：从知识库获取相关信息"""
    print("\n--- RETRIEVE CONTEXT ---")
    last_message = state["messages"][-1].content
    
    # 执行检索
    docs = retriever.invoke(last_message)
    context = format_docs(docs)
    
    # 计算Token节省
    orig_token_count = sum(len(encoding.encode(doc.page_content)) for doc in docs)
    context_token_count = len(encoding.encode(context))
    savings = orig_token_count - context_token_count
    
    print(f"🔍 检索到 {len(docs)} 条相关文档")
    # print(f"📉 Token节省: {savings} (原始: {orig_token_count} -> 压缩: {context_token_count})")
    print(f"📝 注入上下文:\n{context[:300]}...")
    
    return {"context": context}

def generate_with_context(state: SelectStrategyState):
    """生成节点：使用检索到的上下文生成回答"""
    print("\n--- GENERATE WITH CONTEXT ---")
    question = state["messages"][-1].content
    
    # 使用RAG链生成回答
    response = rag_chain.invoke(question)
    
    # 创建消息对象
    response_message = HumanMessage(content=response)
    
    # 输出结果
    print(f"💡 生成的回答: {response}")
    return {"messages": [response_message]}


In [15]:
# --- 4. 构建选择策略图 ---
from langgraph.graph import StateGraph

# 定义状态
class SelectState(TypedDict):
    messages: Annotated[List[BaseMessage], operator.add]
    context: str

# 创建图
select_graph = StateGraph(SelectState)

# 添加节点
select_graph.add_node("retrieve", retrieve_context)
select_graph.add_node("generate", generate_with_context)

# 设置入口点
select_graph.set_entry_point("retrieve")

# 添加边
select_graph.add_edge("retrieve", "generate")
select_graph.add_edge("generate", END)

# 编译图
select_workflow = select_graph.compile()

In [16]:
# --- 5. 演示选择策略 ---
print("\n### 演示'选择'策略 ###")
question = "请为我们的电力造价管理系统写一封促销公众号短文，突出其核心优势"

# 初始状态
initial_state = SelectState(
    messages=[HumanMessage(content=question)],
    context=""
)

# 执行工作流
for step in select_workflow.stream(initial_state):
    if "__end__" not in step:
        print(step)
        print("---")


### 演示'选择'策略 ###

--- RETRIEVE CONTEXT ---
🔍 检索到 2 条相关文档
📝 注入上下文:
## 来源 1:
电力造价管理系统：
- 理念：电力工程全生命周期造价管理
- 覆盖专业：发电/变电/输电/农配网/运维检修五大领域
- 核心功能：支持可研估算、初步设计概算、施工图预算、招投标报价、竣工结算全流程
- 技术特点：内置行业计价依据与业务场景库
- 优势：智能高效完成造价编制，确保合规性与准确性
- 适用对象：电力工程造价咨询机构、建设单位

## 来源 2:
电力业务管理工具集：
- 核心产品：清标工具（招标文件智能审查）
           智慧招投标系统（全流程电子化）
- 功能亮点：自动识别风险项、智能生成审查报告
- 适用对象：业主单位/施工单位/招标代理机构
- 效...
{'retrieve': {'context': '## 来源 1:\n电力造价管理系统：\n- 理念：电力工程全生命周期造价管理\n- 覆盖专业：发电/变电/输电/农配网/运维检修五大领域\n- 核心功能：支持可研估算、初步设计概算、施工图预算、招投标报价、竣工结算全流程\n- 技术特点：内置行业计价依据与业务场景库\n- 优势：智能高效完成造价编制，确保合规性与准确性\n- 适用对象：电力工程造价咨询机构、建设单位\n\n## 来源 2:\n电力业务管理工具集：\n- 核心产品：清标工具（招标文件智能审查）\n           智慧招投标系统（全流程电子化）\n- 功能亮点：自动识别风险项、智能生成审查报告\n- 适用对象：业主单位/施工单位/招标代理机构\n- 效率提升：人工成本降低50%，错误率下降80%'}}
---

--- GENERATE WITH CONTEXT ---
💡 生成的回答: 【电力造价管理，精准高效，助力工程全程管控】  

在电力工程中，造价管理是关键环节，直接影响项目质量与效益。我们的**电力造价管理系统**，以“全生命周期造价管理”为核心理念，全面覆盖发电、变电、输电、农配网及运维检修五大领域，支持可研估算、初步设计概算、施工图预算、招投标报价、竣工结算等全流程造价工作。  

系统内置权威行业计价依据与丰富业务场景库，智能高效完成造价编制，确保合规性与准确性，大幅提升工作效率。  

无论是电力

## **策略三：压缩 (Compress) - 为上下文"瘦身减负"**

**核心思想：** 使用总结(summarization)和裁剪(trimming)技术减少上下文长度，节省Token并提高效率。

**实现两种压缩技术：**
1. **总结压缩**：将长文本提炼为简洁摘要
2. **裁剪压缩**：智能保留对话中最相关的部分

**场景演示：** 智能体需要阅读一篇长文章并回答问题，我们通过总结压缩文章内容；同时展示对话历史裁剪技术。

In [17]:
# --- 1. 准备长文本示例 ---
long_article = """

一. 总体工作概要
从3月11日进入公司以来, 我的工作开展如下:

AI技术研究为AI项目提供支持能力
客户真实场景需求驱动技术研究迭代
AI知识管理将经验转化为可复用的方法
二. 相关工作情况
2.1. 技术研究
围绕两项技术: 多模态图纸识别 和 RAG(检索增强生成) 开展研究工作.

2.1.1. 多模态图纸识别方面
从少样本提示识别, 到搭建系统流程输出提资表, 再到通过YOLO(目标检测模型)+VL(视觉理解模型)+context(上下文信息)的信息输出, 在多个场景下验证了AI的能力.

完成了少样本识别提示的应用界面Demo(见后附图1). 支持用户将工程图纸上传, 然后根据图纸信息进行定额匹配, 也可根据软件内的定额\预规等, 帮助用户快速生成费用预估. 但是这项技术也存在不足, 主要是少样本提示不能无限扩充, 并且模型的指引跟随具有随机性.
实现了整套图纸识别输出提资表的应用界面Demo(见后附图2). 这项技术可以从整套工程图纸中提取工程造价所需的参数(目前初步实现了架空线路杆塔工程的提取), 最终愿景是用户将设计资料导入后就能够自动生成造价成果. 当然我们距离这个最终愿景还有一定的距离, 需要解决的有识别准确率, 多文件信息的交叉验证等技术问题, 也有大量的业务问题.
搭建了YOLO+VL+context的应用界面Demo(见后附图3). 这项技术研究在一定程度上可以解决上述第2项研究的部分问题, 但更大的价值还是构建了一个具有一定业务通用性的技术平台, 也就是说除了造价, 识图也可以用于如评审 施工等各个领域, 甚至也可以用户非工程领域. 而我们针对性要开展的工作也是更为明确的: 算法岗位负责YOLO训练, 大模型岗位负责VL模型应用系统搭建, 需求岗负责context构建.
2.1.2. RAG(检索增强生成)方面
RAG的研究经过两轮迭代,目前在Index(数据预处理和向量索引构建), Retrieval(检索方法), Generation(结果生成)方面都具备了较为体系化的解决方案.

Index方面, 构建了文档嵌入系统Demo, 提供了灵活可选择的向量索引嵌入方法(见后附图4). 而在数据预处理方面. 并且为了解决原始数据质量不佳的问题, 研究了基于多模态大模型对原始文档进行处理的方法(见后附图5), 针对复杂的文档结构(多栏 图 文 表 公式兼具, 还有页眉页脚信息), 实现了极高的转化准确率, 输出的markdown文本后续chunk 进行RAG入库, 可以显著提升检索准确率, 并可实现按照图片URL的回答渲染(这套方法处于验证可行阶段, 搭建系统还需要一定过程).
Retrieval方面, 实验了多种embedding模型和rerank模型, 并且实践了GLM-SDK Dashscope-SDK Langchain 等多种框架, 以及Faiss Chroma等多种向量数据库, 针对多种场景下的技术选型, 具有了比较成熟的经验.(Retrieval过程没有构建界面展示)
Generation方面, 迭代了两版内容生成界面(见后附图6). 第二版相较于第一版做了思考模式 多轮对话 对话过程记录等方面的关键优化, 可显著提升运用效果.
以上两项技术研究, 我认为是具有生长为两条产品线的潜力的.
2.2. 项目支持
AI项目支持面向的省份主要是 浙江 江苏 广东 这三个发达地区省份

2.2.1. 对浙江支持的情况
上半年以来, 浙江客户对AI项目的迫切度是最高, 3 4 5三个月频繁在浙江温州 嘉兴 绍兴 等地市出差, 针对各地市承担的省公司下方AI建设任务, 也都积极地给出了相关解决方案乃至演示Demo. 但到目前为止, 项目都没有落下来, 我认为没有牵头人是最关键的原因.
客户方没有牵头人. 原本是省公司安排任务, 地市公司执行. 但是省公司只安排任务, 不安排费用, 这就导致地市纷纷观望, 等待省公司再给出明确的发文. 本质上这就是一个谁投入 谁负责 以及谁受益的问题.
而我们本身也是一样.

2.2.2. 对江苏支持的情况
6月份以来, 根据江苏分公司的要求, 对江苏的AI项目进行了支持, 包括江苏省经研院和江苏省咨询公司. 江苏这边由分公司杨正健经理作为对接牵头人, 并且其也能够调用分公司资源开展部分工作. 所以江苏这边项目工作的安排我个人认为是更为有序和合理的.

2.2.3. 对广东支持的情况
广东的支持围绕是广州院的项目, 得益于万可团队对项目推动的主动性, 我在其中的角色是最清晰的(也就是承担技术咨询支持的工作), 而其他相关项目常规动作都由其团队自行完成. 而目前看来, 广东的项目推进进度也是相对较好的.
以上三个省份的支持, 代表了三种模式, 而到底采取什么样的模式, 也是接下来各地AI项目开展要解决的关键问题.

2.3. 客户对接
在对各省AI项目支持, 与客户对接的过程中, 对客户在AI领域的认知和需求, 也形成一定的总结.

2.3.1. 客户对AI的认知
对于大模型的一些基本概念或者能做什么, 客户是有所了解的. 但是由于大多数人接触的都是短视频/推文类的快信息, 所以无法对大模型建立系统性的认识, 即大模型的原理, 提示词工程的内涵, RAG的流程以及模型微调作用机制等, 这些对于客户来说大多无概念。
这就会形成沟通上的不一致, 典型的说法有:"明明这个很简单，你们为什么做不了?"、"这个不可能做到，还需要等一等再说." 这些其实就是客户站在既有的认知框架内, 对大模型能力的个人主观推断所形成的下意识表达。
而我们的销售/市场人员, 也难说有很深度的认知.
所以针对客户要求, 实际验证很重要, 要以实际跑出来的效果对客户提出的要求进行更准确的判断.

2.3.2. 客户对AI的需求
就目前接触到的客户来说, 对AI的需求基本还就是在识图和内容生成方面.
对于识图, 是需要从图中提取到准确的业务参数, 从而推动下一步业务工作的自动化开展. 而内容生成是包括了问答, 文档产出等工作, 以替代原本由人工去查询和编写的低效方式.
对于有关造价软件的直接的AI需求, 目前还尚未明确获取到. 但是我认为这是不可忽视, 并且将是极为重要的一个点, 挖掘我们造价软件的潜藏AI使用需求, 推动软件价值的进一步升级, 是一定要开展的重点工作.

2.4. 知识管理
知识管理方面包括团队分享和个人学习, 在组织的各种形态中, 学习型组织的生命力是最强的.

2.4.1. 团队分享
入司以来的5次AI技术分享:

入司首日分享的 DeepSeek概念及用法
提示词工程入门
RAG基本原理和优化策略
Agent 设计模式
模型训练(已准备好分享资料,暂未开展分享会)
2.4.2. 个人学习
包括对模型 开发框架 工具使用的学习:

DeepSeek Qwen GLM InternVL等系列模型的能力实践
Langchain/Langgraph streamlit 等大模型应用开发框架的学习实践
MCP CherryStudiu Cline等各种AI辅助工具的使用
2.5. 工作情况小结
以上是我入司四个月以来的主要工作情况. 我认为这四个月以来我是践行了重回公司所想做的事情, 也 是在公司 部门 和领导的支持鼓励下, 可以说取得了一定的工作成效. 从只是说AI, 转变到可以做一些具体的事情, 有一些看得到的东西.
但同时, 我也逐步认识到个人的局限性. 随着以上四项工作逐渐做深, 每项工作所需的时间和精力投入都是越来越大. 虽然说能够在AI的辅助下, 尽可能地提升效率, 但也明显感觉越来越力不从心.

三. 计划和建议
3.1. 下半年计划
基于上半年的技术研究基础、项目支持经验以及对客户需求的深入理解, 下半年工作将围绕技术深化、项目落地、价值挖掘和知识赋能四个核心方向展开, 具体计划如下:

3.1.1. 技术研究深化与平台化
3.1.1.1. 多模态图纸识别：
目标是提升工程参数提取的准确率与鲁棒性, 探索1-2个新需求场景的可行性验证. 包括以下工作开展:

优化平台, 重点解决多文件信息交叉验证难题, 建立更完善的业务上下文构建规范与流程.
跟进YOLO模型的小样本训练优化, 提升目标检测精度.
深化VL模型在复杂图纸理解中的应用策略, 探索模型微调对特定任务的增益.
3.1.1.2. RAG技术工程化
目标是完成多模态文档智能处理与RAG集成系统的工程化落地, 显著提升知识库构建效率与问答质量.包括以下工作开展:

将验证成功的多模态文档转Markdown方案工程化, 实现自动化或半自动化的复杂文档（含图/文/表/公式）高质量预处理。
整合文档处理、Chunk策略、向量化方法，构建标准化、可配置的索引构建流水线。
完善Retrieval & Generation, 基于前期框架选型经验, 固化最佳实践, 提升系统响应速度与结果可靠性.
3.1.2. AI项目落地攻坚
目标是推动至少 1个省份的AI项目 实现实质性落地(签订合同或进入明确实施阶段)并优化支持模式, 提升整体项目成功率。

全力配合江苏等能够有较高层次的客户对接人的省份AI项目, 利用已验证的Demo和技术能力, 推动项目需求细化、方案确认与合同落地.
总结江苏（分公司牵头+资源协调）、广东（主动团队+清晰分工）模式的关键要素, 形成内部项目推动指南, 指导后续其他省份AI项目推进.
3.1.3. 挖掘软件AI潜能
启动造价软件核心业务流程的AI需求挖掘与概念验证。

与造价产品、研发团队紧密协作，系统梳理造价软件关键模块中潜在的AI应用点(如智能定额匹配、异常审核提示、报告自动生成).
针对1-2个高价值场景,利用现有技术能力(如少样本提示、RAG)进行快速概念验证, 展示AI提升软件效率与价值的可能性.
3.1.4. 知识管理升级与团队赋能
将个人经验与学习成果有效转化为团队共享资产, 提升团队整体AI能力.

系统整理前期技术研究文档、Demo代码、客户常见问题、最佳实践，构建可检索的团队内部AI知识库.
专题分享与培训: 完成“模型训练”分享, 围绕下半年重点技术(如YOLO优化实践、多模态RAG系统详解、软件AI PoC经验)和项目经验, 组织定期专题分享会. 针对市场团队, 开展客户AI沟通与需求引导专项培训, 提升其应对“认知鸿沟”的能力.
协作机制: 在技术研究项目中(如YOLO+VL平台、RAG系统), 明确跨岗位协作流程与接口(算法、大模型、需求), 促进知识在协作中流动.
3.2. 工作建议
为有效支撑上述计划的达成, 并促进团队在AI领域的长期健康发展, 谨提出以下建议:

建立清晰的技术研究与项目落地协同机制:
明确技术研究团队(或核心人员)与区域销售/项目经理在AI项目中的责任界面. 技术团队聚焦解决方案设计、关键技术攻关与核心Demo交付; 销售/项目经理负责客户关系维护、商务谈判、资源协调及整体项目落地推动. 建立定期项目同步会机制, 确保信息对齐, 技术支撑及时响应项目关键节点.
系统化构建团队AI知识资产与能力:
将知识管理升级为团队级任务. 设立专项资源（可兼职）, 负责持续收集、整理、更新技术文档、案例研究、客户Q&A、最佳实践, 维护易用的内部知识库. 制度化技术分享, 要求参与重点项目的成员进行经验复盘与分享. 为新加入或需提升的成员设计AI认知与沟通培训课程。
探索设立AI创新孵化或预研小组：
考虑组建一个小的、跨职能的虚拟小组(包含研发、产品、技术研究代表), 定期（如双周）聚焦研讨核心软件AI需求挖掘、前沿技术跟踪评估(如Agent、模型微调新进展)、以及高潜力新方向的快速概念验证. 给予小组一定的资源进行小范围探索.
"""

In [19]:
# --- 2. 定义压缩工具 ---
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# 总结压缩工具
summary_prompt = ChatPromptTemplate.from_template(
    "请将以下文本总结为不超过{max_words}字的关键要点，保留所有核心工作内容：\n\n{text}"
)

summarizer_chain = (
    summary_prompt
    | model
    | StrOutputParser()
)

# 裁剪压缩函数
def trim_messages(messages: List[BaseMessage], max_messages=5) -> List[BaseMessage]:
    """裁剪对话历史，保留系统消息和最新的几条消息"""
    # 始终保留第一条系统消息
    system_message = messages[0] if messages and isinstance(messages[0], SystemMessage) else None
    
    # 保留最近的max_messages条消息（排除系统消息）
    recent_messages = messages[-max_messages:] if len(messages) > 1 else messages
    
    # 重新组合
    trimmed = []
    if system_message:
        trimmed.append(system_message)
    trimmed.extend(recent_messages)
    
    return trimmed

In [20]:
# --- 3. 定义状态和节点 ---
class CompressState(TypedDict):
    messages: Annotated[List[BaseMessage], operator.add]
    original_text: str  # 原始长文本
    compressed_text: str  # 压缩后的文本
    token_savings: int  # 节省的Token数量

def compress_long_text(state: CompressState):
    """总结压缩节点：将长文本压缩为摘要"""
    print("\n--- COMPRESSING LONG TEXT ---")
    last_message = state["messages"][-1]
    
    # 从用户消息中提取问题
    question = last_message.content
    
    # 压缩长文本
    summary = summarizer_chain.invoke({"text": state["original_text"], "max_words": 300})
    
    # 计算Token节省
    orig_tokens = len(encoding.encode(state["original_text"]))
    comp_tokens = len(encoding.encode(summary))
    savings = orig_tokens - comp_tokens
    
    print(f"📉 文本压缩: {orig_tokens} tokens → {comp_tokens} tokens (节省 {savings} tokens)")
    print(f"📝 压缩摘要:\n{summary[:200]}...")
    
    # 更新状态
    return {
        "compressed_text": summary,
        "token_savings": savings,
        "messages": [HumanMessage(content=f"基于以下摘要回答问题:\n{summary}\n\n问题: {question}")]
    }

def answer_with_compressed_text(state: CompressState):
    """回答节点：基于压缩文本回答问题"""
    print("\n--- ANSWERING WITH COMPRESSED TEXT ---")
    
    # 调用模型生成答案
    response = model.invoke(state["messages"])
    answer = response.content
    
    print(f"💡 生成的回答: {answer}")
    return {"messages": [response]}

def trim_context(state: CompressState):
    """裁剪节点：压缩对话历史"""
    print("\n--- TRIMMING CONTEXT ---")
    
    # 计算裁剪前的Token
    all_messages = "".join(m.content for m in state["messages"])
    before_tokens = len(encoding.encode(all_messages))
    
    # 执行裁剪
    trimmed_messages = trim_messages(state["messages"], max_messages=3)
    
    # 计算裁剪后的Token
    trimmed_content = "".join(m.content for m in trimmed_messages)
    after_tokens = len(encoding.encode(trimmed_content))
    savings = before_tokens - after_tokens
    
    print(f"✂️ 裁剪历史: {len(state['messages'])}条 → {len(trimmed_messages)}条消息")
    print(f"📉 Token节省: {savings} (原始: {before_tokens} -> 裁剪后: {after_tokens})")
    
    return {"messages": trimmed_messages}

In [21]:
# --- 4. 构建压缩策略图 ---
compress_graph = StateGraph(CompressState)

# 添加节点
compress_graph.add_node("compress", compress_long_text)
compress_graph.add_node("answer", answer_with_compressed_text)
compress_graph.add_node("trim", trim_context)

# 设置入口点
compress_graph.set_entry_point("compress")

# 添加边
compress_graph.add_edge("compress", "answer")
compress_graph.add_edge("answer", END)

# 添加条件边用于裁剪
def should_trim(state: CompressState):
    """当消息超过5条时触发裁剪"""
    if len(state["messages"]) > 5:
        return "trim"
    return END

compress_graph.add_conditional_edges("answer", should_trim, {"trim": "trim", END: END})
compress_graph.add_edge("trim", END)

# 编译图
compress_workflow = compress_graph.compile()

In [22]:
# --- 5. 演示总结压缩 ---
print("\n### 演示'总结压缩'技术 ###")
question = "这段时间做了哪些工作?"

# 初始状态
initial_state = CompressState(
    messages=[SystemMessage(content="你是一个分析师"), HumanMessage(content=question)],
    original_text=long_article,
    compressed_text="",
    token_savings=0
)

# 执行工作流
for step in compress_workflow.stream(initial_state):
    if "__end__" not in step:
        print(step)
        print("---")



### 演示'总结压缩'技术 ###

--- COMPRESSING LONG TEXT ---
📉 文本压缩: 4861 tokens → 421 tokens (节省 4440 tokens)
📝 压缩摘要:
本总结涵盖入职四个月以来的核心工作内容，共分五大部分：

1. **总体工作概要**：围绕AI技术研究、客户场景驱动、知识管理三方面展开，推动AI能力落地。

2. **技术研究**：重点推进多模态图纸识别与RAG技术。前者完成少样本识别、整套图纸提资表及YOLO+VL+context系统Demo，后者实现数据预处理、检索优化与生成界面迭代，具备产品化潜力。

3. **项目支持**：聚焦浙江、江...
{'compress': {'compressed_text': '本总结涵盖入职四个月以来的核心工作内容，共分五大部分：\n\n1. **总体工作概要**：围绕AI技术研究、客户场景驱动、知识管理三方面展开，推动AI能力落地。\n\n2. **技术研究**：重点推进多模态图纸识别与RAG技术。前者完成少样本识别、整套图纸提资表及YOLO+VL+context系统Demo，后者实现数据预处理、检索优化与生成界面迭代，具备产品化潜力。\n\n3. **项目支持**：聚焦浙江、江苏、广东三省，分别体现“无牵头人”、“有协调人”、“主动团队”三种模式，项目落地仍需解决机制问题。\n\n4. **客户对接**：客户对AI认知有限，需求集中在识图与内容生成，未来应挖掘造价软件的AI应用潜力。\n\n5. **知识管理**：组织多次技术分享，个人学习模型开发、框架使用等技能，推动学习型组织建设。\n\n**下半年计划**包括技术深化、项目落地、软件AI潜能挖掘和知识赋能，建议建立协同机制、知识资产体系及创新孵化小组，以支撑长期发展。', 'token_savings': 4440, 'messages': [HumanMessage(content='基于以下摘要回答问题:\n本总结涵盖入职四个月以来的核心工作内容，共分五大部分：\n\n1. **总体工作概要**：围绕AI技术研究、客户场景驱动、知识管理三方面展开，推动AI能力落地。\n\n2. **技术研究**：重点推进多模态图纸识别与RAG技术。前者完成少样本识别、整套图纸提资表及YOLO+VL+contex

In [23]:
# --- 6. 演示对话历史裁剪 ---
print("\n### 演示'裁剪压缩'技术 ###")

# 创建一个长对话历史
long_chat_history = [
    SystemMessage(content="您已接入电力工程设计软件客服助手"),
    HumanMessage(content="你好，我想了解电力设计协同平台支持哪些设计专业？"),
    HumanMessage(content="需要导入国网典型设计方案时，系统有预置模板吗？"),
    HumanMessage(content="标准物料库更新后，如何同步到现有项目？"),
    HumanMessage(content="可视化图形平台是否支持三维设计功能？"),
    HumanMessage(content="设计成果能否直接生成符合行业标准的PDF报告？"),
    HumanMessage(content="初次使用的话，有没有提供试用版本？"),
    HumanMessage(content="技术支持是7x24小时响应吗？服务等级怎么划分？"),
    HumanMessage(content="如果需要定制开发，费用是怎么计算的？"),
    HumanMessage(content="项目完成后，是否提供项目报告？"),
    HumanMessage(content="请问如何获取最新的产品手册和用户指南？"),
    HumanMessage(content="我对电力造价管理系统的核心功能很感兴趣，请给我详细介绍。"),
    HumanMessage(content="请给我详细介绍电力造价管理系统的核心功能。")
]

# 初始状态（无文本压缩）
trim_demo_state = CompressState(
    messages=long_chat_history,
    original_text="",
    compressed_text="",
    token_savings=0
)

# 执行裁剪
trimmed_state = trim_context(trim_demo_state)

# 显示裁剪效果
print("\n裁剪前消息:")
for i, msg in enumerate(long_chat_history):
    prefix = "🤖" if isinstance(msg, SystemMessage) else "👤"
    print(f"{prefix} {msg.content[:50]}{'...' if len(msg.content) > 50 else ''}")

print("\n裁剪后消息:")
for i, msg in enumerate(trimmed_state['messages']):
    prefix = "🤖" if isinstance(msg, SystemMessage) else "👤"
    print(f"{prefix} {msg.content[:50]}{'...' if len(msg.content) > 50 else ''}")



### 演示'裁剪压缩'技术 ###

--- TRIMMING CONTEXT ---
✂️ 裁剪历史: 13条 → 4条消息
📉 Token节省: 169 (原始: 250 -> 裁剪后: 81)

裁剪前消息:
🤖 您已接入电力工程设计软件客服助手
👤 你好，我想了解电力设计协同平台支持哪些设计专业？
👤 需要导入国网典型设计方案时，系统有预置模板吗？
👤 标准物料库更新后，如何同步到现有项目？
👤 可视化图形平台是否支持三维设计功能？
👤 设计成果能否直接生成符合行业标准的PDF报告？
👤 初次使用的话，有没有提供试用版本？
👤 技术支持是7x24小时响应吗？服务等级怎么划分？
👤 如果需要定制开发，费用是怎么计算的？
👤 项目完成后，是否提供项目报告？
👤 请问如何获取最新的产品手册和用户指南？
👤 我对电力造价管理系统的核心功能很感兴趣，请给我详细介绍。
👤 请给我详细介绍电力造价管理系统的核心功能。

裁剪后消息:
🤖 您已接入电力工程设计软件客服助手
👤 请问如何获取最新的产品手册和用户指南？
👤 我对电力造价管理系统的核心功能很感兴趣，请给我详细介绍。
👤 请给我详细介绍电力造价管理系统的核心功能。


## **策略四：隔离 (Isolate) - "分而治之"的架构智慧**

**核心思想：** 将复杂任务分解为多个子任务，由专门的智能体在隔离环境中处理，避免上下文污染。

**实现多智能体架构：** 创建专家智能体团队（分析师+文案）

**场景演示：** 用户上传销售数据CSV，要求分析销售冠军并撰写营销文案。

In [24]:
# --- 1. 准备数据 ---
import pandas as pd
from io import StringIO

# 创建示例销售数据CSV
sales_data = """
日期,产品,销售额,销售量
2024-01-01,电力造价管理系统V3.0,320000,4  
2024-01-01,电力设计协同平台Pro,240000,3   
2024-01-01,易数大数据分析平台,180000,2     
2024-01-01,电力管理工具集标准版,95000,5  
2024-01-02,电力造价管理系统V3.0,280000,3.5
2024-01-02,电力设计协同平台Pro,220000,2.75
2024-01-02,易数大数据分析平台,170000,1.89
2024-01-02,电力管理工具集标准版,88000,4.7
2024-01-03,电力造价管理系统V3.0,360000,4.5
2024-01-03,电力设计协同平台Pro,260000,3.25
2024-01-03,易数大数据分析平台,190000,2.11
2024-01-03,电力管理工具集标准版,102000,5.3
2024-01-04,电力造价管理系统V3.0,300000,3.75
2024-01-04,电力设计协同平台Pro,230000,2.88
2024-01-04,易数大数据分析平台,175000,1.94
2024-01-04,电力管理工具集标准版,92000,4.85
2024-01-05,电力造价管理系统V3.0,400000,5   
2024-01-05,电力设计协同平台Pro,320000,4 
2024-01-05,易数大数据分析平台,220000,2.44
2024-01-05,电力管理工具集标准版,110000,5.5
"""

# 保存为CSV文件
with open("sales_data.csv", "w") as f:
    f.write(sales_data)

In [25]:
# --- 2. 定义状态 ---
# 定义多智能体协作的状态
class AgentState(TypedDict):
    messages: Annotated[List[BaseMessage], operator.add]
    task: str
    analysis_result: str
    final_output: str
    next_agent: str

In [26]:
# --- 3. 定义工具 ---
@tool
def analyze_sales_data(question: str) -> str:
    """
    分析销售数据CSV文件，找出销售额最高的产品及其销售总额。
    参数:
        question (str): 用户的原始问题，用于记录分析背景。
    返回:
        str: 一个逗号分隔的字符串，包含产品名称和总销售额，例如 "产品A,150000"。
    """
    print(f"\n--- TOOL: ANALYZE SALES DATA ---")
    print(f"📝 分析任务: {question}")
    
    try:
        df = pd.read_csv("sales_data.csv")
        product_sales = df.groupby("产品")["销售额"].sum()
        top_product = product_sales.idxmax()
        top_sales = product_sales.max()
        result = f"{top_product},{top_sales}"
        print(f"🏆 分析结果: {result}")
        return result
    except Exception as e:
        return f"分析失败: {e}"

@tool
def write_marketing_copy(product: str, key_points: str) -> str:
    """
    为指定产品撰写营销文案。
    参数:
        product (str): 需要撰写文案的产品名称。
        key_points (str): 文案需要围绕的核心卖点。
    返回:
        str: 生成的营销文案。
    """
    print(f"\n--- TOOL: WRITE MARKETING COPY ---")
    print(f"📝 撰写文案: {product} - {key_points[:50]}...")
    
    writer_prompt = ChatPromptTemplate.from_template(
        "你是一个专业营销文案。请基于以下产品信息撰写一篇不超过150字的吸引人的营销文案:\n"
        "产品名称: {product}\n"
        "核心卖点: {key_points}\n"
        "文案:"
    )
    writer_chain = writer_prompt | model | StrOutputParser()
    
    return writer_chain.invoke({"product": product, "key_points": key_points})

In [27]:
# --- 4. 创建智能体 ---

# Helper function to create a specialist agent
def create_agent(system_prompt: str, tools: list):
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("placeholder", "{messages}"),
    ])
    agent = prompt | model.bind_tools(tools)
    return agent

# 分析师智能体
analyst_agent = create_agent(
    "你是一名专业的数据分析师。你的任务是分析给定的数据并返回关键结果。请使用`analyze_sales_data`工具来完成任务。",
    [analyze_sales_data]
)

# 文案智能体
writer_agent = create_agent(
    "你是一名专业的营销文案。你的任务是根据分析结果，为产品撰写引人注目的营销文案。请使用`write_marketing_copy`工具来完成任务。",
    [write_marketing_copy]
)

In [29]:
# --- 5. 定义智能体节点 ---

def analyst_node(state: AgentState):
    print("\n--- CALLING ANALYST AGENT ---")
    result = analyst_agent.invoke({"messages": [HumanMessage(content=state['task'])]})
    return {"messages": [result]}

def writer_node(state: AgentState):
    print("\n--- CALLING WRITER AGENT ---")
    # 从state中提取分析结果，并作为输入传递给文案智能体
    product, sales = state['analysis_result'].split(',')
    prompt = f"分析结果：销售冠军是‘{product}’，总销售额为 {sales} 元。请为此产品撰写营销文案。"
    result = writer_agent.invoke({"messages": [HumanMessage(content=prompt)]})
    return {"messages": [result]}

# 定义工具执行节点
tool_node = ToolNode([analyze_sales_data, write_marketing_copy])

def execute_tools(state: AgentState):
    print("\n--- EXECUTING TOOLS ---")
    last_message = state['messages'][-1]
    tool_call = last_message.tool_calls[0]
    
    # 执行工具
    tool_result = tool_node.invoke([last_message])
    
    # 根据工具更新状态
    if tool_call['name'] == 'analyze_sales_data':
        return {"messages": tool_result, "analysis_result": tool_result[0].content}
    elif tool_call['name'] == 'write_marketing_copy':
        return {"messages": tool_result, "final_output": tool_result[0].content}
    
    return {"messages": tool_result}

In [30]:
# --- 6. 构建图 (Supervisor模式) ---

def supervisor_router(state: AgentState):
    """路由：决定下一个应该由哪个智能体来处理"""
    print("\n--- SUPERVISOR ---")

    # 如果分析结果还未产生，则分配给分析师
    if not state.get("analysis_result"):
        print("📋 任务分配: 分析师 (Analyst)")
        return "analyst"
        
    # 如果分析已完成但文案还未撰写，则分配给文案
    if state.get("analysis_result") and not state.get("final_output"):
        print("📋 任务分配: 文案撰写 (Writer)")
        return "writer"
        
    # 如果一切都完成了
    print("✅ 所有任务完成")
    return END

# 构建图
isolate_graph = StateGraph(AgentState)

isolate_graph.add_node("analyst", analyst_node)
isolate_graph.add_node("writer", writer_node)
isolate_graph.add_node("execute_tools", execute_tools)

# 设置入口点
isolate_graph.set_entry_point("analyst")

# 定义图的边
isolate_graph.add_edge("analyst", "execute_tools")
isolate_graph.add_edge("writer", "execute_tools")
isolate_graph.add_conditional_edges(
    "execute_tools",
    supervisor_router,
    {"analyst": "analyst", "writer": "writer", END: END}
)

# 编译工作流
isolate_workflow = isolate_graph.compile()

In [31]:
# --- 7. 执行多智能体协作 ---
print("\n### 演示多智能体协作 (Supervisor模式) ###")
task = (
    "分析销售数据找出销售额最高的产品，"
    "然后为该产品撰写一篇吸引人的营销文案。"
)
initial_state = AgentState(
    messages=[],
    task=task,
    analysis_result="",
    final_output="",
    next_agent="analyst"
)

# 执行工作流
for step in isolate_workflow.stream(initial_state, {"recursion_limit": 10}):
    node = list(step.keys())[0]
    state = step[node]
    print(f"--- [{node}] 步骤完成 ---")
    if "final_output" in state and state["final_output"]:
        print(f"\n🎉 最终文案:\n{state['final_output']}")


### 演示多智能体协作 (Supervisor模式) ###

--- CALLING ANALYST AGENT ---
--- [analyst] 步骤完成 ---

--- EXECUTING TOOLS ---

--- TOOL: ANALYZE SALES DATA ---
📝 分析任务: 分析销售数据找出销售额最高的产品，然后为该产品撰写一篇吸引人的营销文案。
🏆 分析结果: 电力造价管理系统V3.0,1660000

--- SUPERVISOR ---
📋 任务分配: 文案撰写 (Writer)
--- [execute_tools] 步骤完成 ---

--- CALLING WRITER AGENT ---
--- [writer] 步骤完成 ---

--- EXECUTING TOOLS ---

--- TOOL: WRITE MARKETING COPY ---
📝 撰写文案: 电力造价管理系统V3.0 - 销售冠军，总销售额为1660000元...

--- SUPERVISOR ---
✅ 所有任务完成
--- [execute_tools] 步骤完成 ---

🎉 最终文案:
电力造价管理系统V3.0，销售冠军，总销售额突破166万元！精准计算、高效管理，助力企业降本增效。智能报价、数据可视化，让造价更轻松。选择V3.0，成就卓越工程！
